# Advent of code

# day 1

In [4]:
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""

1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet

""".strip("\n")
)
f = open("  day01.txt  ".strip())

ans = 0
for l in f:
    numbers = [c for c in l if c.isdigit()]
    ans += int(f"{numbers[0]}{numbers[-1]}")

print(f"""
The sum of all calibration values is {ans}
""".strip())

print()

f = StringIO(
"""

two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen

""".strip("\n")
)
f = open("  day01.txt  ".strip())

ans = 0
for l in f:
    ll = (
        l
        .replace("one", "one 1 one")
        .replace("two", "two 2 two")
        .replace("three", "three 3 three")
        .replace("four", "four 4 four")
        .replace("five", "five 5 five")
        .replace("six", "six 6 six")
        .replace("seven", "seven 7 seven")
        .replace("eight", "eight 8 eight")
        .replace("nine", "nine 9 nine")
        .replace(" ", "")
    )
    numbers = [c for c in ll if c.isdigit()]
    ans += int(f"{numbers[0]}{numbers[-1]}")

print(f"""
The sum of all calibration values in part 2 is {ans}
""".strip())


The sum of all calibration values is 53386

The sum of all calibration values in part 2 is 53312


# day 2

In [18]:
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
""".strip("\n")
)
f = open("  day02.txt  ".strip())

data = []
for l in f:
    nr, game = l.strip().split(":")
    nr = nr[4:]
    gameplays = list(map(
        lambda x: dict([
            ("game", nr),
            *map(
                lambda y: tuple(reversed(y.strip().split(" "))),
                x.split(","))
        ]),
        game.split(";")
    ))
    data += gameplays

data = pd.DataFrame(data).fillna("0").astype(int)

ans = (
    data
    .groupby("game", as_index=False)
    .max()
    .loc[lambda x: (x["red"] <= 12) & (x["green"] <= 13) & (x["blue"] <= 14)]
    ["game"]
    .sum()
)


print(f"""
The sum of the game ids is {ans}
""".strip())

print()

ans = (
    data
    .groupby("game", as_index=False)
    .max()
    .assign(power=lambda x: x["red"] * x["green"] * x["blue"])
    ["power"]
    .sum()
)


print(f"""
The sum of powers of the games is {ans}
""".strip())

The sum of the game ids is 2331

The sum of powers of the games is 71585


# day 3

In [7]:

from io import StringIO
from functools import reduce


import pandas as pd
import numpy as np

f = StringIO(
"""

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

""".strip("\n")
)
f = open("  day03.txt  ".strip())

schematic = np.array(
    list(map(
        lambda x: list(x.strip()),
        f.readlines())),
    dtype=str,
)

x = reduce(np.logical_or, (schematic == str(c) for c in range(10))).astype(int)

y = np.add.accumulate(1 - x, axis=1)
#display(y)
#display((np.concatenate(([0], np.add.accumulate(1 + y[:-1,-1]))) + 1).reshape((-1, 1)))
y = y + (np.concatenate(([0], np.add.accumulate(1 + y[:-1,-1]))) + 1).reshape((-1, 1))
#display(y)
y = y * x

z = 1 - (schematic == ".").astype(int) - x
z = z + np.concatenate((0 * z[:,[-1]], z[:,:-1]), axis=1) + np.concatenate((z[:,1:], 0 * z[:,[0]]), axis=1)
z = z + np.concatenate((0 * z[[-1],:], z[:-1,:]), axis=0) + np.concatenate((z[1:,:], 0 * z[[0],:]), axis=0)
z = np.minimum(z, 1)

w = set((z * y).flatten()) - {0}

acc = {}
for a, b in zip(y.flatten(), schematic.flatten()):
    if a in w:
        acc[a] = acc.get(a, []) + [b]

numbers = {}
for a, b in acc.items():
    numbers[a] = int("".join(b))
    
print(f"""
The sum of the partial numbers is {sum(numbers.values())}
""".strip())

ans = 0
for i, j in np.argwhere(schematic == "*"):
    if len(set(y[max(i-1,0):i+2,max(j-1,0):j+2].flatten())) == 3:
        a, b = tuple(set(y[max(i-1,0):i+2,max(j-1,0):j+2].flatten()) - {0})
        ans += numbers[a] * numbers[b]

print()
print(f"""
The sum gears is {ans}
""".strip())
        

The sum of the partial numbers is 559667

The sum gears is 86841457


# day 4

In [20]:

from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

""".strip("\n")
)
f = open("  day04.txt  ".strip())

ans = 0
for line in f:
    _, a = line.strip().split(":")
    left, right = a.split("|")
    left = set(map(int, left.strip().replace("  ", " ").replace("  ", " ").replace("  ", " ").split(" ")))
    right = list(map(int, right.strip().replace("  ", " ").replace("  ", " ").replace("  ", " ").split(" ")))
    ans += int(2 ** (len([c for c in right if c in left]) - 1))

print(f"""
The total points are {ans}
""".strip())

print()

f = StringIO(
"""

Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11

""".strip("\n")
)
f = open("  day04.txt  ".strip())

count_cards = np.ones(1000, dtype=int)
ncards = 0
for line in f:
    _, a = line.strip().split(":")
    left, right = a.split("|")
    left = set(map(int, left.strip().replace("  ", " ").replace("  ", " ").replace("  ", " ").split(" ")))
    right = list(map(int, right.strip().replace("  ", " ").replace("  ", " ").replace("  ", " ").split(" ")))
    points = len([c for c in right if c in left])
    if points > 0:
        count_cards[ncards+1:ncards+1+points] += count_cards[ncards]
    ncards += 1

print(f"""
The total number of cards is {np.sum(count_cards[:ncards])}
""".strip())


The total points are 21959

The total number of cards is 5132675


# day 5

In [ ]:
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

""".strip("\n")
)
# f = open("  day05.txt  ".strip())



print(f"""
The solution is {"SOLUTION"}
""".strip())

# day 6

In [37]:

import re
from io import StringIO

import pandas as pd
import numpy as np

f = StringIO(
"""

Time:      7  15   30
Distance:  9  40  200

""".strip("\n")
)
f = open("  day06.txt  ".strip())
times = map(int, re.sub(r"\s+", " ", f.readline().split(":")[1].strip()).split(" "))
records = map(int, re.sub(r"\s+", " ", f.readline().split(":")[1].strip()).split(" "))

ans = 1
for t, r in zip(times, records):
    a = 0
    b = (t + 1) // 2
    if t <= 1:
        ans *= 0
    elif r == 0:
        ans *= t - 1
    elif t == 2:
        ans *= 0
    elif r >= b * (t - b):
        ans *= 0
    else:
        while b - a > 1:
            c = (b + a) // 2
            if r >= c * (t - c):
                a = c
            else:
                b = c
        
        lower = b
        
        a = t // 2
        b = t
        while b - a > 1:
            c = (b + a) // 2
            if r < c * (t - c):
                a = c
            else:
                b = c
        
        upper = a
        
        ans *= upper - lower + 1

print(f"""
The solution is {ans}
""".strip())

print()

f = StringIO(
"""

Time:      7  15   30
Distance:  9  40  200

""".strip("\n")
)
f = open("  day06.txt  ".strip())
t = int(re.sub(r"\s", "", f.readline().split(":")[1].strip()))
r = int(re.sub(r"\s", "", f.readline().split(":")[1].strip()))

ans = 1
a = 0
b = (t + 1) // 2
if t <= 1:
    ans *= 0
elif r == 0:
    ans *= t - 1
elif t == 2:
    ans *= 0
elif r >= b * (t - b):
    ans *= 0
else:
    while b - a > 1:
        c = (b + a) // 2
        if r >= c * (t - c):
            a = c
        else:
            b = c

    lower = b

    a = t // 2
    b = t
    while b - a > 1:
        c = (b + a) // 2
        if r < c * (t - c):
            a = c
        else:
            b = c

    upper = a

    ans *= upper - lower + 1



print(f"""
The solution is {ans}
""".strip())

The solution is 2269432

The solution is 35865985


# day 7

In [15]:

from io import StringIO
from collections import Counter

import pandas as pd
import numpy as np

f = StringIO(
"""

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483

""".strip("\n")
)
f = open("  day07.txt  ".strip())

ordered_cards = "A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, 2".split(", ")[::-1]
hands = []
for line in f:
    hand, bid = line.strip().split(" ")
    hands.append((
        *sorted(Counter(hand).values(), reverse=True), 
        *(ordered_cards.index(c) for c in hand),
        int(bid),
    ))

ans = sum((i * j[-1] for i, j in enumerate(sorted(hands), start=1)))

print(f"""
The solution is {ans}
""".strip())

print()

f = StringIO(
"""

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483

""".strip("\n")
)
f = open("  day07.txt  ".strip())

ordered_cards = "A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J".split(", ")[::-1]
hands = []
for line in f:
    hand, bid = line.strip().split(" ")
    hands.append((
        max(Counter(hand.replace("J", "")).values() or [0]) + hand.count("J"),
        *sorted(Counter(hand.replace("J", "")).values(), reverse=True)[1:], 
        *(ordered_cards.index(c) for c in hand),
        int(bid),
    ))

ans = sum((i * j[-1] for i, j in enumerate(sorted(hands), start=1)))

print(f"""
The solution is {ans}
""".strip())

The solution is 253954294

The solution is 254837398


# template